<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">1 - Importation des librairies et chargement des fichiers</h2>
</div>

In [651]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from scipy.stats import kruskal
from scipy.stats import boxcox

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Chargements des fichiers</h3>
</div>

In [652]:
df_pop = pd.read_csv('data/Population_2000_2018.csv')
df_world = pd.read_csv('data/donnees_mondiales_2017.csv')
df_food =pd.read_csv('data/DisponibiliteAlimentaire_2017.csv')
df_internet = pd.read_csv("data/internet.csv",sep=';', encoding='ISO-8859-1')
df_iso = pd.read_csv("data/mapping.csv")

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">2 - Analyse exploratoire des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 - Analyse Pop</h3>
</div>

In [653]:
display(df_pop.head(5))
df_pop.info()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

In [654]:
df_pop = df_pop.drop(['Code Domaine','Domaine','Code zone','Code Élément','Code Produit','Produit','Code année','Unité','Symbole','Description du Symbole','Note'], axis=1)
display(df_pop.head(5))

,Zone,Élément,Année,Valeur
0,Afghanistan,Population totale,2000,20779.953
1,Afghanistan,Population totale,2001,21606.988
2,Afghanistan,Population totale,2002,22600.770
3,Afghanistan,Population totale,2003,23680.871
4,Afghanistan,Population totale,2004,24726.684


In [655]:
df_pop_2017 = df_pop[df_pop['Année'] == 2017]
print(df_pop_2017.head())

              Zone            Élément  Année     Valeur
17     Afghanistan  Population totale   2017  36296.113
36  Afrique du Sud  Population totale   2017  57009.756
55         Albanie  Population totale   2017   2884.169
74         Algérie  Population totale   2017  41389.189
93       Allemagne  Population totale   2017  82658.409


In [656]:
df_pop_2017 = df_pop_2017.rename(columns={'Zone': 'Area'})

In [657]:
df_pivot2 = df_pop_2017.pivot(index='Area', columns=['Élément'], values='Valeur').reset_index()
df_pivot2.columns = ['_'.join(map(str, col)).strip() if isinstance(col, tuple) else col for col in df_pivot2.columns.values]
df_pivot2

,Area,Population totale
0,Afghanistan,36296.113
1,Afrique du Sud,57009.756
2,Albanie,2884.169
3,Algérie,41389.189
4,Allemagne,82658.409
...,...,...
231,Îles Salomon,636.039
232,Îles Turques-et-Caïques,37.115
233,Îles Vierges américaines,104.751
234,Îles Vierges britanniques,29.577


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.2 - Analyse Food</h3>
</div>

In [658]:
df_food.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [659]:
df_food = df_food.drop(['Code Domaine','Domaine','Code zone','Code Élément','Code Produit','Code année','Unité','Symbole','Description du Symbole'], axis=1)
df_food.head()

,Zone,Élément,Produit,Année,Valeur
0,Afghanistan,Production,Blé et produits,2017,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,2017,2302.0
2,Afghanistan,Variation de stock,Blé et produits,2017,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,6701.0


In [660]:
df_Volailles = df_food[df_food['Produit'] == 'Viande de Volailles']

df_Volailles.head()

,Zone,Élément,Produit,Année,Valeur
651,Afghanistan,Production,Viande de Volailles,2017,28.0
652,Afghanistan,Importations - Quantité,Viande de Volailles,2017,29.0
653,Afghanistan,Variation de stock,Viande de Volailles,2017,0.0
654,Afghanistan,Disponibilité intérieure,Viande de Volailles,2017,57.0
655,Afghanistan,Pertes,Viande de Volailles,2017,2.0


In [661]:
df_Volailles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2061 entries, 651 to 176400
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Zone     2061 non-null   object 
 1   Élément  2061 non-null   object 
 2   Produit  2061 non-null   object 
 3   Année    2061 non-null   int64  
 4   Valeur   2061 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 96.6+ KB


In [662]:
df_pivot1 = df_Volailles.pivot(index='Zone', columns=['Élément', 'Produit'], values='Valeur').reset_index()
df_pivot1.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivot1.columns.values]
df_pivot1

,Zone_,Production_Viande de Volailles,Importations - Quantité_Viande de Volailles,Variation de stock_Viande de Volailles,Disponibilité intérieure_Viande de Volailles,Pertes_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Alimentation pour touristes_Viande de Volailles,Traitement_Viande de Volailles,Autres utilisations (non alimentaire)_Viande de Volailles,Aliments pour animaux_Viande de Volailles,Semences_Viande de Volailles
0,Afghanistan,28.0,29.0,0.0,57.0,2.0,0.0,55.0,1.53,5.0,0.54,0.33,NaN,NaN,NaN,NaN,NaN,NaN
1,Afrique du Sud,1667.0,514.0,-0.0,2118.0,83.0,0.0,2035.0,35.69,143.0,14.11,9.25,63.0,0.0,NaN,NaN,NaN,NaN
2,Albanie,13.0,38.0,4.0,47.0,NaN,0.0,47.0,16.36,85.0,6.26,6.45,0.0,NaN,NaN,NaN,NaN,NaN
3,Algérie,275.0,2.0,0.0,277.0,13.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,0.0,NaN,NaN,NaN,NaN
4,Allemagne,1514.0,842.0,-29.0,1739.0,NaN,-38.0,1609.0,19.47,71.0,7.96,4.16,646.0,NaN,167.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Émirats arabes unis,48.0,433.0,-26.0,412.0,NaN,-0.0,412.0,43.47,147.0,14.80,9.25,94.0,NaN,NaN,NaN,NaN,NaN
168,Équateur,340.0,0.0,-1.0,341.0,17.0,0.0,324.0,19.31,83.0,6.15,6.35,0.0,0.0,NaN,NaN,NaN,NaN
169,États-Unis d'Amérique,21914.0,123.0,80.0,18266.0,NaN,-0.0,18100.0,55.68,219.0,19.93,14.83,3692.0,NaN,77.0,89.0,NaN,NaN
170,Éthiopie,14.0,1.0,0.0,14.0,1.0,0.0,14.0,0.13,0.0,0.04,0.03,NaN,0.0,NaN,NaN,NaN,NaN


In [663]:
df_pivot1.isnull().mean()

Zone_                                                                                0.000000
Production_Viande de Volailles                                                       0.023256
Importations - Quantité_Viande de Volailles                                          0.011628
Variation de stock_Viande de Volailles                                               0.017442
Disponibilité intérieure_Viande de Volailles                                         0.011628
Pertes_Viande de Volailles                                                           0.610465
Résidus_Viande de Volailles                                                          0.046512
Nourriture_Viande de Volailles                                                       0.011628
Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles           0.000000
Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles                   0.000000
Disponibilité de protéines en quantité (g/personne/jour)_Via

In [664]:
colonnes_avant = len(df_pivot1.columns)
df_pivot1 = df_pivot1.dropna(axis=1, thresh=int(len(df_pivot1) * 0.7))
colonnes_apres = len(df_pivot1.columns)
print(f"Le DataFrame a été nettoyé de {colonnes_avant - colonnes_apres} colonnes.")

Le DataFrame a été nettoyé de 6 colonnes.


In [665]:
df_pivot1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 12 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Zone_                                                                              172 non-null    object 
 1   Production_Viande de Volailles                                                     168 non-null    float64
 2   Importations - Quantité_Viande de Volailles                                        170 non-null    float64
 3   Variation de stock_Viande de Volailles                                             169 non-null    float64
 4   Disponibilité intérieure_Viande de Volailles                                       170 non-null    float64
 5   Résidus_Viande de Volailles                                                        164 non-null    float64

In [666]:
df_pivot1 = df_pivot1.rename(columns={'Zone_': 'Area'})

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.3 - Analyse WB</h3>
</div>

In [667]:
df_world

,Area,PIB par habitant,Population totale,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Dette publique (% du PIB),Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification
0,Africa Eastern and Southern,1485.753579,632746570.0,35.358901,7.123032,6.399343,NaN,61.891132,46.422853,40.199898
1,Africa Western and Central,1590.555785,431138704.0,46.039478,4.204170,1.725486,NaN,76.413081,66.304602,48.789457
2,Arab World,6068.603008,423664839.0,58.371290,11.068716,2.069329,NaN,4.375673,26.893261,90.666302
3,Caribbean small states,11576.276969,3016651.0,46.320181,10.961184,1.518207,NaN,13.463039,18.051597,95.743033
4,Central Europe and the Baltics,14225.877325,102740078.0,62.248455,5.297647,2.075936,NaN,16.467021,4.443614,99.869143
...,...,...,...,...,...,...,...,...,...,...
261,Virgin Islands (U.S.),35365.069304,107281.0,95.603000,11.955000,NaN,NaN,5.300000,NaN,100.000000
262,West Bank and Gaza,3620.360487,4454805.0,75.894000,25.677000,0.212571,NaN,10.500000,14.700000,99.900000
263,"Yemen, Rep.",893.716501,30034389.0,36.016000,18.564000,NaN,NaN,3.000000,36.800000,79.200000
264,Zambia,1495.752138,17298054.0,42.976000,8.518000,6.577312,52.280942,83.500000,44.000000,40.300000


In [668]:
df_world.isnull().mean()

Area                               0.000000
PIB par habitant                   0.030075
Population totale                  0.003759
Taux d'urbanisation                0.011278
Taux de chômage                    0.116541
Taux d'inflation                   0.139098
Dette publique (% du PIB)          0.729323
Part des énergies renouvelables    0.022556
Taux de mortalité infantile        0.082707
Taux d'électrification             0.011278
dtype: float64

In [669]:
colonnes_avant = len(df_world.columns)
colonnes_manquantes = df_world.columns[df_world.isnull().mean() > 0.3]

print(colonnes_manquantes)


df_world = df_world.dropna(axis=1, thresh=int(len(df_world) * 0.7))

colonnes_apres = len(df_world.columns)
print(f"Le DataFrame a été nettoyé de {colonnes_avant - colonnes_apres} colonnes.")

Index(['Dette publique (% du PIB)'], dtype='object')
Le DataFrame a été nettoyé de 1 colonnes.


In [670]:
df_world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Area                             266 non-null    object 
 1   PIB par habitant                 258 non-null    float64
 2   Population totale                265 non-null    float64
 3   Taux d'urbanisation              263 non-null    float64
 4   Taux de chômage                  235 non-null    float64
 5   Taux d'inflation                 229 non-null    float64
 6   Part des énergies renouvelables  260 non-null    float64
 7   Taux de mortalité infantile      244 non-null    float64
 8   Taux d'électrification           263 non-null    float64
dtypes: float64(8), object(1)
memory usage: 18.8+ KB


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.4 - Analyse internet</h3>
</div>

In [671]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  264 non-null    object 
 1   Country Code  266 non-null    object 
 2   2017          253 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.4+ KB


In [672]:
df_internet = df_internet.rename(columns={'Country Name': 'Area', '2017': 'Internet'})

In [673]:
df_internet['Internet'] = pd.to_numeric(df_internet['Internet'], errors='coerce')

In [674]:
df_internet.head()

,Area,Country Code,Internet
0,Aruba,ABW,97.170000
1,NaN,AFE,17.310987
2,Afghanistan,AFG,13.500000
3,NaN,AFW,27.081219
4,Angola,AGO,26.000000


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">3 - Fusion</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.1 - Verification des pays avant de les fusionner</h3>
</div>

In [675]:
for df in [df_world, df_pivot1, df_pivot2, df_internet]:
    # Convertir les noms des pays en minuscules
    df['Area'] = df['Area'].astype(str).apply(lambda x: x.strip().lower())

C:\Users\ZAPHAS\AppData\Local\Temp\ipykernel_5472\1688907807.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Area'] = df['Area'].astype(str).apply(lambda x: x.strip().lower())


In [676]:
# Uniformiser les noms de pays
df_iso['Country'] = df_iso['Country'].str.strip().str.lower()
df_iso['PAYS'] = df_iso['PAYS'].str.strip().str.lower()

In [677]:
print("Unique values in df_world['Area']:", df_world['Area'].unique())
print("Unique values in df_pivot1['Area']:", df_pivot1['Area'].unique())
print("Unique values in df_pivot2['Area']:", df_pivot2['Area'].unique())
print("Unique values in df_internet['Area']:", df_internet['Area'].unique())
print("Unique values in df_iso['Country']:", df_iso['Country'].unique())
print("Unique values in df_iso['PAYS']:", df_iso['PAYS'].unique())

Unique values in df_world['Area']: ['africa eastern and southern' 'africa western and central' 'arab world'
 'caribbean small states' 'central europe and the baltics'
 'early-demographic dividend' 'east asia & pacific'
 'east asia & pacific (excluding high income)'
 'east asia & pacific (ida & ibrd countries)' 'euro area'
 'europe & central asia' 'europe & central asia (excluding high income)'
 'europe & central asia (ida & ibrd countries)' 'european union'
 'fragile and conflict affected situations'
 'heavily indebted poor countries (hipc)' 'high income' 'ibrd only'
 'ida & ibrd total' 'ida blend' 'ida only' 'ida total'
 'late-demographic dividend' 'latin america & caribbean'
 'latin america & caribbean (excluding high income)'
 'latin america & the caribbean (ida & ibrd countries)'
 'least developed countries: un classification' 'low & middle income'
 'low income' 'lower middle income' 'middle east & north africa'
 'middle east & north africa (excluding high income)'
 'middle east & 

In [678]:
df_iso

,Unnamed: 0,Country,Alpha-3 code,PAYS,3 lettres
0,0,afghanistan,AFG,afghanistan,AFG
1,1,albania,ALB,albanie,ALB
2,2,algeria,DZA,algérie,DZA
3,3,andorra,AND,andorre,AND
4,4,angola,AGO,angola,AGO
...,...,...,...,...,...
211,211,wallis and futuna,WLF,wallis et futuna (îles),WLF
212,212,western sahara,ESH,sahara occidental,ESH
213,213,yemen,YEM,yemen,YEM
214,214,zambia,ZMB,zambie,ZMB


In [679]:
regions_to_exclude = [
    'afrique du nord et moyen-orient', 
    'afrique du nord et moyen-orient (bird et ida)', 
    'afrique du nord et moyen-orient (hors revenu élevé)',
    'afrique subsaharienne', 
    'afrique subsaharienne (bird et ida)', 
    'afrique subsaharienne (hors revenu élevé)',
    'amérique du nord', 
    'amérique latine et caraïbes', 
    'amérique latine et caraïbes (bird et ida)', 
    'amérique latine et caraïbes (hors revenu élevé)',
    'asie de l’est et pacifique', 
    'asie de l’est et pacifique (bird et ida)', 
    'asie de l’est et pacifique (hors revenu élevé)', 
    'asie du sud', 
    'asie du sud (bird et ida)', 
    'autres petits états', 
    'bird et ida', 
    'bird seulement', 
    'de dividende précoce démographique', 
    'de dividende tardif démographique',
    'de post-dividende démographique', 
    'de pré-dividende démographique', 
    'europe centrale et les pays baltes',
    'europe et asie centrale', 
    'europe et asie centrale (bird et ida)', 
    'europe et asie centrale (hors revenu élevé)', 
    'faible revenu', 
    'fragile et les situations de conflit touchées', 
    'le monde arabe', 
    'non classifié', 
    'not classified', 
    'pays les moins avancés: classement de l’onu', 
    "pays membres de l'ocde", 
    'pays pauvres très endettés (ppte)', 
    'petits états', 
    'petits états des caraïbes', 
    'petits états insulaires du pacifique', 
    'revenu faible et intermédiaire', 
    'revenu intermédiaire', 
    'revenu intermédiaire, tranche inférieure', 
    'revenu intermédiaire, tranche supérieure', 
    'revenu élevé', 
    'zone euro', 
    'monde',
    'africa eastern and southern',
    'africa western and central',
    'arab world',
    'caribbean small states',
    'central europe and the baltics',
    'east asia & pacific',
    'east asia & pacific (excluding high income)',
    'east asia & pacific (ida & ibrd countries)',
    'euro area',
    'europe & central asia',
    'europe & central asia (excluding high income)',
    'europe & central asia (ida & ibrd countries)',
    'european union',
    'fragile and conflict affected situations',
    'heavily indebted poor countries (hipc)',
    'high income',
    'ibrd only',
    'ida & ibrd total',
    'ida blend',
    'ida only',
    'late-demographic dividend',
    'latin america & caribbean',
    'latin america & caribbean (excluding high income)',
    'latin america & the caribbean (ida & ibrd countries)',
    'least developed countries: un classification',
    'low & middle income',
    'low income',
    'lower middle income',
    'middle east & north africa',
    'middle east & north africa (excluding high income)',
    'middle east & north africa (ida & ibrd countries)',
    'middle income',
    'north america',
    'oecd members',
    'other small states',
    'pacific island small states',
    'pre-demographic dividend',
    'post-demographic dividend',
    'small states',
    'south asia',
    'south asia (ida & ibrd)',
    'sub-saharan africa',
    'sub-saharan africa (excluding high income)',
    'sub-saharan africa (ida & ibrd countries)',
    'upper middle income',
    'world'
]


# Dictionnaire de correspondance pour harmoniser les noms des pays
country_mapping = {
    "afghanistan": ["afghanistan"],
    "south africa": ["afrique du sud", "south africa"],
    "albania": ["albanie", "albania"],
    "algeria": ["algérie", "algeria"],
    "germany": ["allemagne", "germany"],
    "angola": ["angola"],
    "argentina": ["argentine", "argentina"],
    "armenia": ["arménie", "armenia"],
    "australia": ["australie", "australia"],
    "austria": ["autriche", "austria"],
    "azerbaijan": ["azerbaïdjan", "azerbaijan"],
    "bahamas": ["bahamas", "bahamas, the"],
    "bangladesh": ["bangladesh"],
    "belgium": ["belgique", "belgium"],
    "belize": ["belize"],
    "benin": ["bénin", "benin"],
    "bhutan": ["bhoutan", "bhutan"],
    "bolivia": ["bolivia", "bolivie (état plurinational de)"],
    "bosnia and herzegovina": ["bosnie-herzégovine", "bosnia and herzegovina"],
    "botswana": ["botswana"],
    "brazil": ["brésil", "brazil"],
    "bulgaria": ["bulgarie", "bulgaria"],
    "burkina faso": ["burkina faso"],
    "belarus": ["bélarus", "belarus"],
    "cape verde": ["cabo verde", "cape verde"],
    "cambodia": ["cambodge", "cambodia"],
    "cameroon": ["cameroun", "cameroon"],
    "canada": ["canada"],
    "ivory coast": ["côte d'ivoire", "ivory coast"],
    "chile": ["chili", "chile"],
    "china": ["chine, continentale", "china"],
    "cyprus": ["chypre", "cyprus"],
    "colombia": ["colombie", "colombia"],
    "congo, dem. rep.": ["république démocratique du congo", "congo, dem. rep.", "congo, république démocratique du"],
    "congo, rep.": ["république du congo", "congo, rep.", "congo"],
    "cuba": ["cuba"],
    "denmark": ["danemark", "denmark"],
    "djibouti": ["djibouti"],
    "dominican republic": ["dominicaine (république)", "dominican republic"],
    "ecuador": ["équateur", "ecuador"],
    "egypt": ["égypte", "egypt", "égypte, république arabe d'", "egypt, arab rep."],
    "estonia": ["estonie", "estonia"],
    "eswatini": ["eswatini"],
    "ethiopia": ["éthiopie", "ethiopia"],
    "fiji": ["fidji", "fiji"],
    "finland": ["finlande", "finland"],
    "france": ["france"],
    "gabon": ["gabon"],
    "gambia": ["gambie", "gambia", "gambia, the"],
    "georgia": ["géorgie", "georgia"],
    "ghana": ["ghana"],
    "greece": ["grèce", "greece"],
    "guatemala": ["guatemala"],
    "guinea": ["guinée", "guinea"],
    "guinea-bissau": ["guinée-bissau", "guinea-bissau"],
    "guyana": ["guyana"],
    "honduras": ["honduras"],
    "hungary": ["hongrie", "hungary"],
    "iceland": ["islande", "iceland"],
    "india": ["inde", "india"],
    "indonesia": ["indonésie", "indonesia"],
    "iran": ["iran", "iran (république islamique d')", "iran, islamic rep."],
    "iraq": ["iraq"],
    "ireland": ["irlande", "ireland"],
    "israel": ["israël", "israel"],
    "italy": ["italie", "italy"],
    "jamaica": ["jamaïque", "jamaica"],
    "japan": ["japon", "japan"],
    "jordan": ["jordanie", "jordan"],
    "kazakhstan": ["kazakhstan"],
    "kenya": ["kenya"],
    "south korea": ["corée (république de)", "south korea", "république de corée", "korea, rep."],
    "north korea": ["corée, république démocratique de", "north korea", "république populaire démocratique de corée", "korea, dem. people's rep."],
    "kuwait": ["koweït", "kuwait"],
    "lebanon": ["liban", "lebanon"],
    "lesotho": ["lesotho"],
    "liberia": ["libéria", "liberia"],
    "libya": ["libye", "libya"],
    "lithuania": ["lituanie", "lithuania"],
    "luxembourg": ["luxembourg"],
    "madagascar": ["madagascar"],
    "malawi": ["malawi"],
    "malaysia": ["malaisie", "malaysia"],
    "mali": ["mali"],
    "malta": ["malte", "malta"],
    "mexico": ["mexique", "mexico"],
    "monaco": ["monaco"],
    "morocco": ["maroc", "morocco"],
    "mozambique": ["mozambique"],
    "myanmar": ["myanmar"],
    "namibia": ["namibie", "namibia"],
    "nepal": ["népal", "nepal"],
    "netherlands": ["pays-bas", "netherlands"],
    "new zealand": ["nouvelle-zélande", "new zealand"],
    "niger": ["niger"],
    "nigeria": ["nigéria", "nigeria"],
    "norway": ["norvège", "norway"],
    "oman": ["oman"],
    "pakistan": ["pakistan"],
    "panama": ["panama"],
    "paraguay": ["paraguay"],
    "peru": ["pérou", "peru"],
    "philippines": ["philippines"],
    "poland": ["pologne", "poland"],
    "portugal": ["portugal"],
    "qatar": ["qatar"],
    "romania": ["roumanie", "romania"],
    "russia": ["fédération de russie", "russian federation"],
    "rwanda": ["rwanda"],
    "saudi arabia": ["arabie saoudite", "saudi arabia"],
    "senegal": ["sénégal", "senegal"],
    "serbia": ["serbie", "serbia"],
    "sierra leone": ["sierra leone"],
    "singapore": ["singapour", "singapore"],
    "slovakia": ["slovaquie", "slovak republic", "slovakia"],
    "slovenia": ["slovénie", "slovenia"],
    "somalia": ["somalie", "somalia"],
    "south sudan": ["soudan du sud", "south sudan"],
    "spain": ["espagne", "spain"],
    "sri lanka": ["sri lanka"],
    "sudan": ["soudan", "sudan"],
    "sweden": ["suède", "sweden"],
    "switzerland": ["suisse", "switzerland"],
    "syria": ["république arabe syrienne", "syrian arab republic", "syria"],
    "taiwan": ["taïwan", "taiwan", "chine, taiwan province de"],
    "tanzania": ["tanzanie", "tanzania", "république-unie de tanzanie"],
    "thailand": ["thaïlande", "thailand"],
    "togo": ["togo"],
    "trinidad and tobago": ["trinité-et-tobago", "trinidad and tobago"],
    "tunisia": ["tunisie", "tunisia"],
    "turkey": ["turquie", "turkey", "turkiye"],
    "uganda": ["ouganda", "uganda"],
    "ukraine": ["ukraine"],
    "united arab emirates": ["émirats arabes unis", "united arab emirates"],
    "united kingdom": ["royaume-uni de grande-bretagne et d'irlande du nord", "united kingdom", "royaume-uni"],
    "united states": ["états-unis d'amérique", "united states", "états-unis"],
    "uruguay": ["uruguay"],
    "venezuela": ["venezuela", "venezuela, rb", "venezuela (république bolivarienne du)"],
    "vietnam": ["viet nam", "vietnam"],
    "zambia": ["zambie", "zambia"],
    "zimbabwe": ["zimbabwe"],
    "andorra": ["andorre", "andorra"],
    "anguilla": ["anguilla"],
    "bahrain": ["bahreïn", "bahrain"],
    "bermuda": ["bermudes", "bermuda"],
    "british virgin islands": ["îles vierges britanniques", "british virgin islands"],
    "croatia": ["croatie", "croatia"],
    "eritrea": ["érythrée", "eritrea"],
    "french guiana": ["guyane française", "french guiana"],
    "gibraltar": ["gibraltar"],
    "guadeloupe": ["guadeloupe"],
    "martinique": ["martinique"],
    "micronesia, fed. sts.": ["micronésie (états fédérés de)", "micronesia, fed. sts."],
    "montserrat": ["montserrat"],
    "palestine": ["palestine"],
    "polynesia": ["polynésie française", "polynesia"],
    "saint kitts and nevis": ["saint-kitts-et-nevis", "saint kitts and nevis", "st. kitts and nevis"],
    "saint lucia": ["sainte-lucie", "saint lucia", "st. lucia"],
    "saint vincent and the grenadines": ["saint-vincent-et-les grenadines", "saint vincent and the grenadines", "st. vincent and the grenadines"],
    "saint-barthélemy": ["saint-barthélemy"],
    "saint martin (french part)": ["saint-martin (partie française)", "saint martin (french part)", "st. martin (french part)"],
    "sint maarten (dutch part)": ["sint maarten (partie néerlandaise)", "sint maarten (dutch part)"],
    "western sahara": ["sahara occidental", "western sahara"],
    "yemen": ["yémen", "yemen", "yemen, rep.", "yémen, rép. du"],
    "wallis and futuna": ["îles wallis-et-futuna", "wallis and futuna"],
    "falkland islands (malvinas)": ["îles falkland (malvinas)", "falkland islands (malvinas)"],
    "cayman islands": ["îles caïmanes", "cayman islands"],
    "tokelau": ["tokélaou", "tokelau"],
    "czechia": ["tchéquie", "czechia"],
    "costa rica": ["costa rica"],
    "equatorial guinea": ["guinée équatoriale", "equatorial guinea"],
    "papua new guinea": ["papouasie-nouvelle-guinée", "papua new guinea"],
    "macao sar, china": ["chine - ras de macao", "macao sar, china", "région administrative spéciale de macao, chine"],
    "hong kong sar, china": ["chine - ras de hong-kong", "hong kong sar, china", "chine, ras de hong kong"],
    "palau": ["palaos", "palau"],
    "antigua and barbuda": ["antigua and barbuda", "antigua-et-barbuda"],
    "bahamas": ["bahamas", "bahamas, the"],
    "bolivia": ["bolivia", "bolivie", "bolivie (état plurinational de)"],
    "british virgin islands": ["îles vierges britanniques", "british virgin islands"],
    "cape verde": ["cabo verde", "cape verde"],
    "china": ["chine", "chine, continentale", "china"],
    "comoros": ["comores", "comoros"],
    "ivory coast": ["côte d'ivoire", "ivory coast"],
    "curacao": ["curaçao", "curacao"],
    "guinea": ["guinée", "guinea"],
    "haiti": ["haïti", "haiti"],
    "hong kong sar, china": ["chine - ras de hong-kong", "hong kong sar, china"],
    "macao sar, china": ["région administrative spéciale de macao, chine", "macao sar, china"],
    "marshall islands": ["îles marshall", "marshall islands"],
    "micronesia, fed. sts.": ["micronésie (états fédérés de)", "micronesia, fed. sts."],
    "saint kitts and nevis": ["saint-kitts-et-nevis", "saint kitts and nevis", "st. kitts and nevis"],
    "saint lucia": ["sainte-lucie", "saint lucia", "st. lucia"],
    "saint vincent and the grenadines": ["saint-vincent-et-les grenadines", "saint vincent and the grenadines", "st. vincent and the grenadines"],
    "saint martin (french part)": ["saint-martin (partie française)", "saint martin (french part)", "st. martin (french part)"],
    "south korea": ["corée (république de)", "south korea", "république de corée", "korea, rep."],
    "north korea": ["corée, république démocratique de", "north korea", "république populaire démocratique de corée", "korea, dem. people's rep."],
    "iran": ["iran", "iran (république islamique d')", "iran, islamic rep."],
    "equatorial guinea": ["guinée équatoriale", "equatorial guinea"],
    "papua new guinea": ["papouasie-nouvelle-guinée", "papua new guinea"],
    "brunei darussalam" : ["brunéi darussalam", "brunei darussalam"],
    "cayman islands" : ["îles caïmanes", "cayman islands"],
    "central african republic" : ["république centrafricaine", "central african republic"],
    "comoros" : ["comores", "comoros"],
    "curacao" : ["curaçao", "curacao"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
    "dominica" : ["dominique", "dominica"],
    "faroe islands" : ["îles féroé", "faroe islands"],
    "gibraltar" : ["gibraltar"],
    "greenland" : ["groenland", "greenland"],
    "grenada" : ["grenade", "grenada"],
    "guam" : ["guam"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "hong kong sar, china" : ["chine - ras de hong-kong", "hong kong sar, china", "chine, ras de hong kong"],
    "isle of man" : ["île de man", "isle of man"],
    "kiribati" : ["kiribati"],
    "kosovo" : ["kosovo"],
    "kyrgyz republic" : ["kirghizistan", "kyrgyz republic"],
    "lao pdr" : ["république démocratique populaire lao", "lao pdr"],
    "liechtenstein" : ["liechtenstein"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "macao sar, china" : ["région administrative spéciale de macao, chine", "macao sar, china"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "maldives" : ["maldives"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "marshall islands" : ["îles marshall", "marshall islands"],
    "micronesia, fed. sts." : ["micronésie (états fédérés de)", "micronesia, fed. sts."],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "montserrat" : ["montserrat"],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    "palestine" : ["palestine"], 
    "congo" : ["congo", "congo, république du","congo, dem. rep.","république du congo", "congo, rep.", "congo"],
    "cuba" : ["cuba"],
    "denmark" : ["danemark", "denmark"],
    "djibouti" : ["djibouti"],
    "dominican republic" : ["dominicaine (république)", "dominican republic"],
    "ecuador" : ["équateur", "ecuador"],
    "egypt" : ["égypte", "egypt", "égypte, république arabe d'", "egypt, arab rep."],
    "estonia" : ["estonie", "estonia"],
    "eswatini" : ["eswatini"],
    "ethiopia" : ["éthiopie", "ethiopia"],
    "fiji" : ["fidji", "fiji"],
    "finland" : ["finlande", "finland"],
    "france" : ["france"],
    "gabon" : ["gabon"],
    "gambia" : ["gambie", "gambia", "gambia, the"],
    "georgia" : ["géorgie", "georgia"],
    "ghana" : ["ghana"],
    "greece" : ["grèce", "greece"],
    "guatemala" : ["guatemala"],
    "guinea" : ["guinée", "guinea"],
    "guinea-bissau" : ["guinée-bissau", "guinea-bissau"],
}





# Fonction pour harmoniser les noms des pays et exclure les régions
def harmonize_country_names_and_exclude_regions(df, country_mapping, column_name):
    # Création d'un dictionnaire plat pour les remplacements
    mapping = {v: k for k, v_list in country_mapping.items() for v in v_list}
    
    # Remplacer les noms des pays en utilisant le dictionnaire de correspondance
    df[column_name] = df[column_name].replace(mapping)
    
    # Supprimer les lignes dont les noms appartiennent à la liste des régions à exclure
    df = df[~df[column_name].isin(regions_to_exclude)]
    
    return df

# Harmonisation des noms des pays et suppression des régions dans chaque DataFrame
df_world = harmonize_country_names_and_exclude_regions(df_world, country_mapping, 'Area')
df_pivot1 = harmonize_country_names_and_exclude_regions(df_pivot1, country_mapping, 'Area')
df_pivot2 = harmonize_country_names_and_exclude_regions(df_pivot2, country_mapping, 'Area')
df_internet = harmonize_country_names_and_exclude_regions(df_internet, country_mapping, 'Area')

C:\Users\ZAPHAS\AppData\Local\Temp\ipykernel_5472\3150019218.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(mapping)


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.2 - Fusion</h3>
</div>

In [680]:
df_merge_1 = df_world.merge(df_pivot1, on='Area', how='outer', indicator=True)
print(df_merge_1['_merge'].value_counts())
df_merge_1.drop(columns='_merge', inplace=True)

df_merge_2 = df_merge_1.merge(df_pivot2, on='Area', how='outer', indicator=True)
print(df_merge_2['_merge'].value_counts())
df_merge_2.drop(columns='_merge', inplace=True)


df_final = df_merge_2.merge(df_internet, on='Area', how='outer', indicator=True)
print(df_final['_merge'].value_counts())

df_final.drop(columns='_merge', inplace=True)

_merge
both          153
left_only      66
right_only     20
Name: count, dtype: int64
_merge
both          208
left_only      31
right_only     29
Name: count, dtype: int64
_merge
both          206
left_only      62
right_only     22
Name: count, dtype: int64


In [681]:
del df_final['Country Code']

In [682]:
df_final

,Area,PIB par habitant,Population totale_x,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification,Production_Viande de Volailles,...,Disponibilité intérieure_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Population totale_y,Internet
0,afghanistan,526.140801,35643418.0,25.250,11.184,4.975952,19.5,51.5,97.7,28.0,...,57.0,0.0,55.0,1.53,5.0,0.54,0.33,NaN,36296.113,13.500000
1,albania,4531.032207,2873457.0,59.383,13.616,1.986661,36.9,8.3,99.9,13.0,...,47.0,0.0,47.0,16.36,85.0,6.26,6.45,0.0,2884.169,62.400000
2,algeria,4615.868744,41136546.0,72.052,12.000,5.591116,0.1,21.0,99.5,275.0,...,277.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,41389.189,47.691055
3,american samoa,12372.884783,49463.0,87.170,NaN,NaN,0.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,andorra,40632.231552,73837.0,88.150,NaN,NaN,18.0,3.1,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.001,91.567467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,îles mariannes du nord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.562,NaN
286,îles salomon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,3.0,0.0,3.0,4.45,18.0,1.51,1.31,0.0,636.039,11.924229
287,îles turques-et-caïques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.115,NaN
288,îles vierges (eu),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.377494


In [683]:
df_final_clean = df_final.groupby('Area', as_index=False).mean()

In [684]:
print("Unique values in df_final['Area']:", df_final_clean['Area'].unique())

Unique values in df_final['Area']: ['afghanistan' 'albania' 'algeria' 'american samoa' 'andorra' 'angola'
 'anguilla' 'antigua and barbuda' 'antilles néerlandaises (ex)'
 'argentina' 'armenia' 'aruba' 'asie de l\x92est et pacifique'
 'asie de l\x92est et pacifique (bird et ida)'
 'asie de l\x92est et pacifique (hors revenu élevé)' 'australia' 'austria'
 'azerbaijan' 'bahamas' 'bahrain' 'bangladesh' 'barbade' 'barbados'
 'belarus' 'belgium' 'belize' 'benin' 'bermuda' 'bhutan' 'bolivia'
 'bonaire, saint-eustache et saba' 'bosnia and herzegovina' 'botswana'
 'brazil' 'british virgin islands' 'brunei darussalam' 'bulgaria'
 'burkina faso' 'burundi' 'cambodia' 'cameroon' 'canada' 'cape verde'
 'cayman islands' 'central african republic' 'chad' 'channel islands'
 'chile' 'china' 'chine - ras de macao' 'cisjordanie et gaza' 'colombia'
 'comoros' 'congo' 'congo, dem. rep.' 'corée, république de' 'costa rica'
 "cote d'ivoire" 'croatia' 'cuba' 'curacao' 'cyprus' 'czechia' 'denmark'
 'djibouti' '

In [685]:
df_final_clean.head()

,Area,PIB par habitant,Population totale_x,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification,Production_Viande de Volailles,...,Disponibilité intérieure_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Population totale_y,Internet
0,afghanistan,526.140801,35643418.0,25.250,11.184,4.975952,19.5,51.5,97.7,28.0,...,57.0,0.0,55.0,1.53,5.0,0.54,0.33,NaN,36296.113,13.500000
1,albania,4531.032207,2873457.0,59.383,13.616,1.986661,36.9,8.3,99.9,13.0,...,47.0,0.0,47.0,16.36,85.0,6.26,6.45,0.0,2884.169,62.400000
2,algeria,4615.868744,41136546.0,72.052,12.000,5.591116,0.1,21.0,99.5,275.0,...,277.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,41389.189,47.691055
3,american samoa,12372.884783,49463.0,87.170,NaN,NaN,0.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,andorra,40632.231552,73837.0,88.150,NaN,NaN,18.0,3.1,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.001,91.567467


In [686]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 22 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Area                                                                               290 non-null    object 
 1   PIB par habitant                                                                   212 non-null    float64
 2   Population totale_x                                                                219 non-null    float64
 3   Taux d'urbanisation                                                                217 non-null    float64
 4   Taux de chômage                                                                    189 non-null    float64
 5   Taux d'inflation                                                                   183 non-null    float64

In [687]:
df_final = df_final.dropna(axis=0, thresh=int(df_final.shape[1] * 0.99))

In [688]:
df_final

,Area,PIB par habitant,Population totale_x,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification,Production_Viande de Volailles,...,Disponibilité intérieure_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Population totale_y,Internet
0,afghanistan,526.140801,35643418.0,25.250,11.184,4.975952,19.5,51.5,97.7,28.0,...,57.0,0.0,55.0,1.53,5.0,0.54,0.33,NaN,36296.113,13.500000
1,albania,4531.032207,2873457.0,59.383,13.616,1.986661,36.9,8.3,99.9,13.0,...,47.0,0.0,47.0,16.36,85.0,6.26,6.45,0.0,2884.169,62.400000
2,algeria,4615.868744,41136546.0,72.052,12.000,5.591116,0.1,21.0,99.5,275.0,...,277.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,41389.189,47.691055
5,angola,2439.374439,30208628.0,64.839,16.617,29.844480,52.5,53.4,42.9,42.0,...,319.0,0.0,315.0,10.56,35.0,3.60,2.22,0.0,29816.766,26.000000
9,argentina,14613.035649,44044811.0,91.749,8.347,NaN,10.4,9.8,100.0,2161.0,...,1962.0,0.0,1856.0,42.24,182.0,13.53,13.84,207.0,43937.140,74.294907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,uruguay,18995.397020,3422200.0,95.240,7.890,6.218094,60.8,6.8,99.8,33.0,...,33.0,0.0,31.0,9.12,33.0,2.97,2.23,3.0,3436.641,70.322354
271,vanuatu,3032.197030,290239.0,25.163,5.288,3.084526,35.7,17.5,62.8,1.0,...,3.0,0.0,3.0,11.66,40.0,4.05,2.48,NaN,285.510,30.800000
273,vietnam,2992.071532,94033048.0,35.213,1.874,3.520257,28.3,17.2,100.0,918.0,...,1206.0,0.0,1166.0,12.33,48.0,4.01,3.40,1.0,94600.648,58.140000
279,zambia,1495.752138,17298054.0,42.976,8.518,6.577312,83.5,44.0,40.3,49.0,...,60.0,0.0,58.0,3.42,11.0,1.15,0.72,1.0,16853.599,12.200000


In [689]:
len(country_mapping)

195

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.3 - Remplir les NaN</h3>
</div>

In [690]:
from sklearn.impute import KNNImputer

# Initialisation de l'imputeur KNN avec k voisins
imputer = KNNImputer(n_neighbors=5)

# Imputation sur les données numériques
df_final_numeric = df_final.select_dtypes(include=[float])

# Appliquer KNN Imputer sur les colonnes numériques uniquement
df_final_imputed = pd.DataFrame(imputer.fit_transform(df_final_numeric), columns=df_final_numeric.columns)

# Réinjecter les valeurs imputées dans le DataFrame original
df_final.update(df_final_imputed)


In [691]:
df_final

,Area,PIB par habitant,Population totale_x,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification,Production_Viande de Volailles,...,Disponibilité intérieure_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Population totale_y,Internet
0,afghanistan,526.140801,35643418.0,25.250,11.184,4.975952,19.5,51.5,97.7,28.0,...,57.0,0.0,55.0,1.53,5.0,0.54,0.33,248.6,36296.113,13.500000
1,albania,4531.032207,2873457.0,59.383,13.616,1.986661,36.9,8.3,99.9,13.0,...,47.0,0.0,47.0,16.36,85.0,6.26,6.45,0.0,2884.169,62.400000
2,algeria,4615.868744,41136546.0,72.052,12.000,5.591116,0.1,21.0,99.5,275.0,...,277.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,41389.189,47.691055
5,angola,4041.995071,2851923.0,63.103,12.968,0.969553,12.6,11.7,99.7,11.0,...,47.0,0.0,47.0,16.06,54.0,5.44,3.39,0.0,2944.791,64.744884
9,argentina,30666.883865,399020.0,82.925,9.800,1.518207,1.1,11.8,100.0,6.0,...,26.0,0.0,16.0,43.17,182.0,14.61,13.33,1.0,381.755,80.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,uruguay,18995.397020,3422200.0,95.240,7.890,6.218094,60.8,6.8,99.8,33.0,...,33.0,0.0,31.0,9.12,33.0,2.97,2.23,3.0,3436.641,70.322354
271,vanuatu,3032.197030,290239.0,25.163,5.288,3.084526,35.7,17.5,62.8,1.0,...,3.0,0.0,3.0,11.66,40.0,4.05,2.48,NaN,285.510,30.800000
273,vietnam,2992.071532,94033048.0,35.213,1.874,3.520257,28.3,17.2,100.0,918.0,...,1206.0,0.0,1166.0,12.33,48.0,4.01,3.40,1.0,94600.648,58.140000
279,zambia,1495.752138,17298054.0,42.976,8.518,6.577312,83.5,44.0,40.3,49.0,...,60.0,0.0,58.0,3.42,11.0,1.15,0.72,1.0,16853.599,12.200000


In [692]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141 entries, 0 to 280
Data columns (total 22 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Area                                                                               141 non-null    object 
 1   PIB par habitant                                                                   141 non-null    float64
 2   Population totale_x                                                                141 non-null    float64
 3   Taux d'urbanisation                                                                141 non-null    float64
 4   Taux de chômage                                                                    140 non-null    float64
 5   Taux d'inflation                                                                   141 non-null    float64
 6  

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">4 - Export des données</h2>
</div>

In [693]:
# Exporter le DataFrame final en CSV
df_final.to_csv('df_final.csv', index=False)

In [ ]:
jupyter nbconvert --to markdown --stdout notebook.ipynb > output_notebook.txt